In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

nvs_prodoctor = pd.DataFrame()

os.chdir(r'C:\Users\AmanovRA\Текущее\Prodoctor\january-juny')
for file in os.listdir('.'):
    if 'Appointment' in file:
        temp_df = pd.read_excel(r'C:\Users\AmanovRA\Текущее\Prodoctor\january-juny' + '\\' + file, skiprows=3)
        temp_df['Источник'] = [file[:-5] for i in temp_df['Пациент']]
        temp_df['Пациент'].apply(int).apply(str)
        nvs_prodoctor = nvs_prodoctor.append(temp_df, ignore_index=True)
        
   
nvs_prodoctor = nvs_prodoctor.loc[:,['Пациент', 'Источник']]
nvs_prodoctor = nvs_prodoctor.rename(columns={'Пациент' : 'Телефон'})
nvs_prodoctor = nvs_prodoctor.drop_duplicates(subset='Телефон')
nvs_prodoctor.reset_index(inplace=True, drop=True)

In [23]:
nvs_prodoctor

,Телефон,Источник
0,79264829144,Appointment-2020-1
1,73833461898,Appointment-2020-1
2,79139537084,Appointment-2020-1
3,79021793232,Appointment-2020-1
4,79289718741,Appointment-2020-1
...,...,...
767,79137389493,Appointment-2020-6
768,79612208064,Appointment-2020-6
769,79134804473,Appointment-2020-6
770,79930236428,Appointment-2020-6


In [33]:
au_telefons = pd.read_excel('NSK_january_juny.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)  

lst_with_coincidenc_telefon = []
lst_with_coincidenc_source = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append([])
    lst_with_coincidenc_source.append([])
    for i, j in enumerate(nvs_prodoctor['Телефон']):
        if str(j) in row:
            lst_with_coincidenc_telefon[index].append(str(j))
            lst_with_coincidenc_source[index].append(nvs_prodoctor['Источник'][i])
            break
            
au_telefons['Prodoctor-телефон'], au_telefons['Prodoctor-источник'] = lst_with_coincidenc_telefon, lst_with_coincidenc_source           
            
def join_list(lst):
    return ', '.join([str(i) for i in lst])

au_telefons['Prodoctor-телефон'], au_telefons['Prodoctor-источник'] = au_telefons['Prodoctor-телефон'].apply(join_list), au_telefons['Prodoctor-источник'].apply(join_list)

In [35]:
au_telefons.to_excel(r'C:\Users\AmanovRA\Текущее\Prodoctor\january-juny\НВС_Prodoctor_по_июнь.xlsx')